In [1]:
import numpy as np

MORSE = {'A':'.-', 'B':'-...', 'C':'-.-.', 'D':'-..', 'E':'.', 'F':'..-.', # ... 생략
         '0':'-----','1':'.----', '2':'..---','3':'...--','4':'....-','5':'.....',
         '6':'-....','7':'--...','8':'---..','9':'----.'}

def synth_morse(text, fs=8000, f0=600, wpm=20, a=0.9, atk=0.005, dcy=0.005):
    # WPM→dit 길이(sec): 표준 근사 T ≈ 1.2 / WPM
    T = 1.2 / wpm
    tone = lambda dur: np.sin(2*np.pi*f0*np.arange(int(dur*fs))/fs)
    env = lambda n: np.concatenate([
        np.linspace(0,1,int(atk*fs),endpoint=False),
        np.ones(max(n - int((atk+dcy)*fs), 0)),
        np.linspace(1,0,int(dcy*fs),endpoint=True)
    ])[:n]  # attack/decay 적용

    sig = []
    for ch in text.upper():
        if ch==' ':
            # 단어 간 공백: 7T (문자간 3T 포함 규약 감안)
            sig.append(np.zeros(int(7*T*fs)))
            continue
        code = MORSE.get(ch)
        if not code: 
            continue
        for i,sym in enumerate(code):
            dur = T if sym=='.' else 3*T
            tone_seg = tone(dur)
            tone_seg = (tone_seg * env(len(tone_seg))) * a
            sig.append(tone_seg)
            # 요소 간 공백 T
            sig.append(np.zeros(int(T*fs)))
        # 문자 간 추가 공백 3T - 마지막 요소 공백 T 보정 → 2T
        sig.append(np.zeros(int(2*T*fs)))
    return np.concatenate(sig) if sig else np.zeros(0)
